In [ ]:
import datetime as dt

In [ ]:
# Just to initialize Python.NET
import orchid

In [ ]:
import dateutil.parser as dup
import toolz.curried as toolz

In [ ]:
# noinspection PyUnresolvedReferences
from System import Type
# noinspection PyUnresolvedReferences
from System.Data import DataTable, DataColumn

In [ ]:
some_data = {'foo': [3, 1, 4],
             'bar': [3.14, 2.718, 1.414],
             'baz': ['abc', 'def', 'ghi'],
             'quux': [dup.isoparse('2314-02-13T17:03:22Z'),
                      dup.isoparse('2718-10-03T08:17:49Z'),
                      dup.isoparse('2141-07-17T10:24:16Z')]}

In [ ]:
type_net_type_names = {type(1): "System.Int32",
                       type(2.0): "System.Double",
                       type('xyz'): "System.String",
                       type(dt.datetime): "System.DateTime",}
def net_type_name(python_value):
    return type_net_type_names(type(python_value))

In [ ]:
def populate_data_table(from_data):
    empty_data_table = DataTable()
    data_table_with_columns = add_data_table_columns(empty_data_table, from_data)
    result = add_data_table_rows(data_table_with_columns, from_data)
    return result

def dump_table(data_table):
    print(f'Columns: {[c.ColumnName for c in built_data_table.Columns]}')
    for i, row in enumerate(data_table.Rows):
        print(f'row {i}: ', end='')
        for cn in [c.ColumnName for c in data_table.Columns]:
            print(f'{cn}: ', end='')
            print(f'{row[cn]}, ', end='')
        print('')
        
def dump_column_details(data_table):
    print(f'Columns: {[(c.ColumnName, str(c.DataType), c.ReadOnly) for c in built_data_table.Columns]}')
    
def add_data_table_columns(data_table, from_data):
    for column_name, row_values in from_data.items():
        new_column = make_data_table_column(column_name, row_values[0])
        data_table.Columns.Add(new_column)
    return data_table
    
def make_data_table_column(column_name, row_value):
    new_column = DataColumn()
    new_column.ColumnName = data_name_to_column_name(column_name)
    new_column.DataType = make_data_column_type(row_value)
    new_column.ReadOnly = True
    return new_column
    
def data_name_to_column_name(data_name):
    mapper = {'foo': 'Fooey',
              'bar': 'Bizarre',
              'baz': 'Beltashazzar',}
    try:
        return mapper[data_name]
    except KeyError:
        if data_name == 'quux':
            return "Quark"
        raise
        
def make_data_column_type(row_value):
    mapper = {int: 'System.Int32',
              float: 'System.Double',
              str: 'System.String'}
    
    sought = type(row_value)
    try:
        return Type.GetType(mapper[sought])
    except KeyError:
        if sought == dt.datetime:
            return Type.GetType('System.DateTime')
        raise

def add_data_table_rows(data_table, from_data):
    for column_name, row_values in from_data.items():
        new_row = make_data_table_row(data_table, column_name, row_values)
        data_table.Rows.Add(new_row)
    return data_table
        
def make_data_table_row(data_table, column_name, row_values):
    data_table_row = data_table.NewRow()
    for row_value in row_values:
        net_column_name = data_name_to_column_name(column_name) 
        if net_column_name != 'Quark':
            data_table_row[net_column_name] = row_value
        else:
            data_table_row[net_column_name] = orchid.net_quantity.as_net_date_time(row_value)
    return data_table_row

In [ ]:
built_data_table = populate_data_table(some_data)
dump_table(built_data_table)

In [ ]:
dump_column_details(built_data_table)

In [ ]:
len(built_data_table.Rows)

In [ ]:
for i, row in enumerate(built_data_table.Rows):
    print(f'row {i}')
    for cn in [c.ColumnName for c in built_data_table.Columns]:
        print(f'{cn}: ', end='')
        print(f'{row[cn]}, ', end='')
    print('')